# Introdução

Este projeto tem como objetivo coletar, armazenar e analisar dados históricos de condições climáticas provenientes de diferentes cidades. Utilizando uma base de dados extraída da **HG Brasil** e salva em banco usando **MongoDB**, a aplicação deverá realizar testes com diferentes algoritmos de *machine learning* para registrar seus resultados e permitir uma comparação de eficiência. Os dados coletados incluem temperatura, umidade, descrição do clima, velocidade do vento, entre outros parâmetros relevantes para estudos meteorológicos e análises estatísticas. 

### Inicialização

Para carregar os módulos necessários, execute a célula abaixo para iniciar as bibliotecas usadas para o funcionamento do projeto.

In [ ]:
import json
import random
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Mm
from docx2pdf import convert

from src.db_config import get_data
from src.config import BACKUP_DB, BACKUP_DIR, DB_TABLE_WEATHER

*A base usada durante o treinamento será carregada na célula abaixo, precisando ser inicializada antes de executar os demais pontos.*

In [ ]:
data = get_data(DB_TABLE_WEATHER)

*O código abaixo permite realizar a validação dos dados duplicados, usado para o ajuste de base.*

In [ ]:
contagem = {}
duplicados = []

for item in data:
    chave = (item["results"]["city"], item["results"]["date"], item["results"]["time"])
    contagem[chave] = contagem.get(chave, 0) + 1

resultado = [list(chave) + [quantidade] for chave, quantidade in contagem.items()]

for item_resultado in [filtro_resultado for filtro_resultado in resultado if filtro_resultado[3] > 1]:
    for item in [filtro for filtro in data if filtro["results"]["city"] == item_resultado[0] and filtro["results"]["date"] == item_resultado[1] and filtro["results"]["time"] == item_resultado[2]]:
        duplicados.append({"Cidade": item["results"]["city"], "Data": item["results"]["date"], "Hora": item["results"]["time"], "ID": item["_id"]})

pd.DataFrame(duplicados)

*O bloco abaixo permite a validação parcial dos dados, podendo ser usado para garantir que eles estão sendo salvos corretamente.*

In [ ]:
registros = []

for item in [filtro for filtro in data if filtro["results"]["date"] == "01/10/2025"]:
    registros.append({"Cidade": item["results"]["city"], "Data": item["results"]["date"], "Hora": item["results"]["time"], "C°": item["results"]["temp"], "Descrição": item["results"]["description"]})

pd.DataFrame(registros)

## Início da análise

Para a análise preditiva das informações coletadas será usado o módulo [**scikit-learn**](https://scikit-learn.org/), que é uma biblioteca de código aberto com diversos algoritmos de aprendizado de máquina disponíveis de forma simplificada e com ferramentas para avaliar sua permormance, sendo as principais métricas:

| Métrica | Descrição |
| :---------- | :---------------------------------------------------------------------------------- |
| `Acurácia` | Proporção de previsões classificadas corretamente sobre o total de previsões.        |
| `Precisão` | Proporção de previsões positivas corretas em relação a todas as previsões positivas. Mostra a capacidade do classificador de não rotular como positiva uma amostra negativa. |
| `Recall` | Proporção de previsões positivas em relação a todas as previsões positivas corretas. Mostra a capacidade do classificador de rotular como positivo apenas o que deveria ser positivo. |
| `F1 Score` | Média harmônica entre precisão e recall. |

Com o objetivo de facilitar no retorno dos dados foram criadas as funções a seguir, que permitem tratar e salvar as informações processadas pelos modelos.

In [ ]:
def formata_percentual(valor: float) -> str:
    """
    Formata um valor decimal em percentual com duas casas decimais.\n
    Args:
        valor (float): Valor decimal a ser formatado.
    Returns:
        str: Valor formatado como percentual.
    """
    return f"{valor:.2%}"

def monta_classificacoes(y_teste: list, resultado: np.ndarray) -> list:
    """
    Monta uma lista de dicionários com as métricas de classificação para cada classe.\n
    Args:
        y_teste (list): Lista de valores reais.
        resultado (np.ndarray): Array de valores previstos pelo modelo.
    Returns:
        list: Lista de dicionários com as métricas de classificação.
    """
    informacoes = classification_report(y_teste, resultado, zero_division=0, output_dict=True)
    classificacoes = [{"target": chave, "precision": round(valor["precision"], 2), "recall": round(valor["recall"], 2), "f1-score": round(valor["f1-score"], 2), "support": int(valor["support"])} for chave, valor in informacoes.items() if chave not in ["accuracy", "macro avg", "weighted avg"]]

    return classificacoes

def salva_matriz_de_confusao(y_teste: list, resultado: np.ndarray, nome_arquivo:str, tamanho: float) -> None:
    """
    Salva a matriz de confusão em um arquivo PNG.\n
    Args:
        y_teste (list): Lista de valores reais.
        resultado (np.ndarray): Array de valores previstos pelo modelo.
        nome_arquivo (str): Nome do arquivo PNG a ser salvo.
        tamanho (float): Tamanho da figura.
    """
    print(f"Acurácia: {formata_percentual(accuracy_score(y_teste, resultado))}")

    cmd = ConfusionMatrixDisplay(confusion_matrix(y_teste, resultado))

    fig, ax = plt.subplots(figsize=(tamanho, tamanho))
    cmd.plot(ax=ax)

    plt.savefig(f"./docs/{nome_arquivo}.png", dpi=300, bbox_inches="tight")

Após a inicialização e a carga dos dados, será feito o seu tratamento para realizar o treinamento nos modelos selecionados. O objetivo será usar o mesmo dataset para realizar previsões sobre a **condição climática** e a **temperatura**.

A cada treinamento realizado será apresentada uma tabela com as métricas explicadas anteriormente e gerada sua respectiva **matriz de confusão**, onde sua diagonal informa as previsões corretas para cada classe e os demais dados na mesma linha mostram em qual outra classe foi classificado o erro.

In [ ]:
# Preparação dos dados
cidades = set()
condicoes = set()
temperaturas = set()

# Lista de amostras para visualização
amostras = []
limite_amostras = 1 if len(data) < 50 else int(len(data) / 50)

# Listas para previsoras e alvos das condições climáticas
condicoes_previsoras = []
condicoes_alvo = []

# Listas para previsoras e alvos das temperaturas
temperaturas_previsoras = []
temperaturas_alvo = []

# Processamento dos dados
for item in data:
    # Cálculo do dia do ano
    tempo_final = datetime.strptime(f"{item["results"]["date"]} {item["results"]["time"]}", "%d/%m/%Y %H:%M")
    tempo_inicial = datetime(tempo_final.year, 1, 1, 0, 0, 0)
    tempo_decorrido = tempo_final - tempo_inicial
    segundos_decorridos = tempo_decorrido.total_seconds()
    dia_do_ano = segundos_decorridos / 86400

    # Coleta de valores únicos
    cidades.add(item["results"]["city"])
    condicoes.add(item["results"]["description"])
    temperaturas.add(item["results"]["temp"])

    # Montagem das listas de previsoras e alvos das condições climáticas
    condicoes_previsoras.append([list(cidades).index(item["results"]["city"]), dia_do_ano, item["results"]["temp"], item["results"]["humidity"], item["results"]["cloudiness"], item["results"]["rain"]])
    condicoes_alvo.append(item["results"]["description"])

    # Montagem das listas de previsoras e alvos das temperaturas
    temperaturas_previsoras.append([list(cidades).index(item["results"]["city"]), dia_do_ano, list(condicoes).index(item["results"]["description"]), item["results"]["humidity"], item["results"]["cloudiness"], item["results"]["rain"]])
    temperaturas_alvo.append(item["results"]["temp"])

    # Coleta de amostras para visualização
    if len(amostras) <= 50 and random.randint(1, limite_amostras) == 1:
        amostras.append({"Cidade": item["results"]["city"], "Dia/Hora": f"{item["results"]["date"]} {item["results"]["time"]}", "Dia do ano": dia_do_ano, "Temperatura": item["results"]["temp"], "Humidade": item["results"]["humidity"], "Nebulosidade": item["results"]["cloudiness"], "Chuva": item["results"]["rain"], "Condição": item["results"]["description"]})

# Criação dos nomes das features e tamanho da matriz de confusão das condições climáticas
feature_names_condicoes=["Cidade", "Dia do ano", "Temperatura", "Humidade", "Nebulosidade", "Chuva"]
tamanho_mc_condicoes = len(condicoes) / 2

# Preparação dos dados para condições climáticas
X_condicoes = np.array(condicoes_previsoras)
y_condicoes = np.array(condicoes_alvo)

# Divisão dos dados em conjuntos de treino e teste para condições climáticas
X_train_condicoes, X_test_condicoes, y_train_condicoes, y_test_condicoes = train_test_split(X_condicoes, y_condicoes, test_size=0.5)

# Criação dos nomes das features e tamanho da matriz de confusão das temperaturas
feature_names_temperaturas=["Cidade", "Dia do ano", "Condição", "Humidade", "Nebulosidade", "Chuva"]
tamanho_mc_temperaturas = len(temperaturas) / 2

# Preparação dos dados para temperaturas
X_temperaturas = np.array(temperaturas_previsoras)
y_temperaturas = np.array(temperaturas_alvo)

# Divisão dos dados em conjuntos de treino e teste para temperaturas
X_train_temperaturas, X_test_temperaturas, y_train_temperaturas, y_test_temperaturas = train_test_split(X_temperaturas, y_temperaturas, test_size=0.5)

# Remoção de arquivos antigos
if os.path.exists("./docs/relatorio.docx"):
    os.remove("./docs/relatorio.docx")

if os.path.exists("./docs/matriz_de_confusao_LR_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_LR_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_DTC_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_DTC_condicoes.png")
if os.path.exists("./docs/arvore_de_decisao_condicoes.png"):
    os.remove("./docs/arvore_de_decisao_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_RFC_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_RFC_condicoes.png")
if os.path.exists("./docs/floresta_aleatoria_condicoes.png"):
    os.remove("./docs/floresta_aleatoria_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_MLPC_adam_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_MLPC_adam_condicoes.png")
if os.path.exists("./docs/perda_do_modelo_MLCP_adam_condicoes.png"):
    os.remove("./docs/perda_do_modelo_MLCP_adam_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_MLPC_lbfgs_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_MLPC_lbfgs_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_SVC_rbf_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_SVC_rbf_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_SVC_linear_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_SVC_linear_condicoes.png")
if os.path.exists("./docs/matriz_de_confusao_KNN_condicoes.png"):
    os.remove("./docs/matriz_de_confusao_KNN_condicoes.png")

if os.path.exists("./docs/matriz_de_confusao_LR_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_LR_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_DTC_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_DTC_temperaturas.png")
if os.path.exists("./docs/arvore_de_decisao_temperaturas.png"):
    os.remove("./docs/arvore_de_decisao_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_RFC_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_RFC_temperaturas.png")
if os.path.exists("./docs/floresta_aleatoria_temperaturas.png"):
    os.remove("./docs/floresta_aleatoria_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_MLPC_adam_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_MLPC_adam_temperaturas.png")
if os.path.exists("./docs/perda_do_modelo_MLCP_adam_temperaturas.png"):
    os.remove("./docs/perda_do_modelo_MLCP_adam_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_MLPC_lbfgs_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_MLPC_lbfgs_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_SVC_rbf_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_SVC_rbf_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_SVC_linear_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_SVC_linear_temperaturas.png")
if os.path.exists("./docs/matriz_de_confusao_KNN_temperaturas.png"):
    os.remove("./docs/matriz_de_confusao_KNN_temperaturas.png")

# Relatório inicial
relatorio = {
    "total_registros": len(data),
    "total_cidades": len(cidades),
    "total_condicoes": len(condicoes),
    "total_temperaturas": len(temperaturas),
    "logistic_regression": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "decision_tree_classifier": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "random_forest_classifier": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "mlp_classifier_adam": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "mlp_classifier_lbfgs": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "support_vector_machine_rbf": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "support_vector_machine_linear": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    },
    "k_neighbors_classifier": {
        "acuracia_condicoes": 0,
        "classificacoes_condicoes": [],
        "acuracia_temperaturas": 0,
        "classificacoes_temperaturas": []
    }
}

pd.DataFrame(amostras)

## Análise de condições climáticas


### Regressão logística

In [ ]:
lr_condicoes = make_pipeline(StandardScaler(), LogisticRegression(max_iter=200))
lr_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_lr_condicoes = lr_condicoes.predict(X_test_condicoes)

classificacao_lr_condicoes = monta_classificacoes(y_test_condicoes, resultado_lr_condicoes)

relatorio["logistic_regression"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_lr_condicoes)),
    "classificacao_condicoes": classificacao_lr_condicoes
})

pd.DataFrame(classificacao_lr_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_lr_condicoes, "matriz_de_confusao_LR_condicoes", tamanho_mc_condicoes)

### Árvore de decisão

In [ ]:
dtc_condicoes = DecisionTreeClassifier()
dtc_plot_condicoes = dtc_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_dtc_condicoes = dtc_condicoes.predict(X_test_condicoes)

classificacao_dtc_condicoes = monta_classificacoes(y_test_condicoes, resultado_dtc_condicoes)

relatorio["decision_tree_classifier"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_dtc_condicoes)),
    "classificacao_condicoes": classificacao_dtc_condicoes
})

pd.DataFrame(classificacao_dtc_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_dtc_condicoes, "matriz_de_confusao_DTC_condicoes", tamanho_mc_condicoes)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), dpi=300)
plot_tree(dtc_plot_condicoes, feature_names=feature_names_condicoes, class_names=list(condicoes), max_depth=5, fontsize=6, ax=ax)

plt.savefig("./docs/arvore_de_decisao_condicoes.png", dpi=300, bbox_inches="tight")

### Floresta aleatória

In [ ]:
rfc_condicoes = RandomForestClassifier(n_jobs=2, random_state=0)
rfc_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_rfc_condicoes = rfc_condicoes.predict(X_test_condicoes)

classificacao_rfc_condicoes = monta_classificacoes(y_test_condicoes, resultado_rfc_condicoes)

relatorio["random_forest_classifier"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_rfc_condicoes)),
    "classificacao_condicoes": classificacao_rfc_condicoes
})

pd.DataFrame(classificacao_rfc_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_rfc_condicoes, "matriz_de_confusao_RFC_condicoes", tamanho_mc_condicoes)

In [ ]:
fig, ax = plt.subplots(ncols=len(condicoes), figsize=(90, 20), dpi=300)

for i in range(0, len(condicoes)):
    plot_tree(rfc_condicoes.estimators_[i], feature_names=feature_names_condicoes, class_names=list(condicoes), max_depth=3, fontsize=6, ax=ax[i])

    ax[i].set_title("Estimador: " + str(i), fontsize=10)

plt.savefig("./docs/floresta_aleatoria_condicoes.png", dpi=300, bbox_inches="tight")

### Multi-layer Perceptron (ADAM)

In [ ]:
mlpc_adam_condicoes = MLPClassifier(hidden_layer_sizes=(20,), max_iter=500, activation="relu")
mlpc_adam_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_mlpc_adam_condicoes = mlpc_adam_condicoes.predict(X_test_condicoes)

classificacao_mlpc_adam_condicoes = monta_classificacoes(y_test_condicoes, resultado_mlpc_adam_condicoes)

relatorio["mlp_classifier_adam"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_mlpc_adam_condicoes)),
    "classificacao_condicoes": classificacao_mlpc_adam_condicoes
})

pd.DataFrame(classificacao_mlpc_adam_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_mlpc_adam_condicoes, "matriz_de_confusao_MLPC_adam_condicoes", tamanho_mc_condicoes)

In [ ]:
plt.plot(mlpc_adam_condicoes.loss_curve_)
plt.title("Perda do modelo")
plt.ylabel("Perda")
plt.xlabel("Iterações")
plt.legend(["Treino"], loc="upper left")
plt.savefig("./docs/perda_do_modelo_MLCP_adam_condicoes.png", dpi=300, bbox_inches="tight")

### Multi-layer Perceptron (LBFGS)

In [ ]:
mlpc_lbfgs_condicoes = MLPClassifier(alpha=0, max_iter=1000, solver="lbfgs")
mlpc_lbfgs_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_mlpc_lbfgs_condicoes = mlpc_lbfgs_condicoes.predict(X_test_condicoes)

classificacao_mlpc_lbfgs_condicoes = monta_classificacoes(y_test_condicoes, resultado_mlpc_lbfgs_condicoes)

relatorio["mlp_classifier_lbfgs"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_mlpc_lbfgs_condicoes)),
    "classificacao_condicoes": classificacao_mlpc_lbfgs_condicoes
})

pd.DataFrame(classificacao_mlpc_lbfgs_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_mlpc_lbfgs_condicoes, "matriz_de_confusao_MLPC_lbfgs_condicoes", tamanho_mc_condicoes)

### SVM - Support Vector Machine (RBF)

In [ ]:
svc_rbf_condicoes = SVC(C=50)
svc_rbf_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_svc_rbf_condicoes = svc_rbf_condicoes.predict(X_test_condicoes)

classificacao_svc_rbf_condicoes = monta_classificacoes(y_test_condicoes, resultado_svc_rbf_condicoes)

relatorio["support_vector_machine_rbf"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_svc_rbf_condicoes)),
    "classificacao_condicoes": classificacao_svc_rbf_condicoes
})

pd.DataFrame(classificacao_svc_rbf_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_svc_rbf_condicoes, "matriz_de_confusao_SVC_rbf_condicoes", tamanho_mc_condicoes)

### SVM - Support Vector Machine (Linear)

In [ ]:
svc_linear_condicoes = SVC(kernel="linear")
svc_linear_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_svc_linear_condicoes = svc_linear_condicoes.predict(X_test_condicoes)

classificacao_svc_linear_condicoes = monta_classificacoes(y_test_condicoes, resultado_svc_linear_condicoes)

relatorio["support_vector_machine_linear"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_svc_linear_condicoes)),
    "classificacao_condicoes": classificacao_svc_linear_condicoes
})

pd.DataFrame(classificacao_svc_linear_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_svc_linear_condicoes, "matriz_de_confusao_SVC_linear_condicoes", tamanho_mc_condicoes)

### K Neighbors

In [ ]:
knn_condicoes = make_pipeline(StandardScaler(), KNeighborsClassifier())
knn_condicoes.fit(X_train_condicoes, y_train_condicoes)

resultado_knn_condicoes = knn_condicoes.predict(X_test_condicoes)

classificacao_knn_condicoes = monta_classificacoes(y_test_condicoes, resultado_knn_condicoes)

relatorio["k_neighbors_classifier"].update({
    "acuracia_condicoes": formata_percentual(accuracy_score(y_test_condicoes, resultado_knn_condicoes)),
    "classificacao_condicoes": classificacao_knn_condicoes
})

pd.DataFrame(classificacao_knn_condicoes)

In [ ]:
salva_matriz_de_confusao(y_test_condicoes, resultado_knn_condicoes, "matriz_de_confusao_KNN_condicoes", tamanho_mc_condicoes)

## Análise de temperaturas


### Regressão logística

In [ ]:
lr_temperaturas = make_pipeline(StandardScaler(), LogisticRegression())
lr_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_lr_temperaturas = lr_temperaturas.predict(X_test_temperaturas)

classificacao_lr_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_lr_temperaturas)

relatorio["logistic_regression"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_lr_temperaturas)),
    "classificacao_temperaturas": classificacao_lr_temperaturas
})

pd.DataFrame(classificacao_lr_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_lr_temperaturas, "matriz_de_confusao_LR_temperaturas", tamanho_mc_temperaturas)

### Árvore de decisão

In [ ]:
dtc_temperaturas = DecisionTreeClassifier()
dtc_plot_temperaturas = dtc_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_dtc_temperaturas = dtc_temperaturas.predict(X_test_temperaturas)

classificacao_dtc_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_dtc_temperaturas)

relatorio["decision_tree_classifier"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_dtc_temperaturas)),
    "classificacao_temperaturas": classificacao_dtc_temperaturas
})

pd.DataFrame(classificacao_dtc_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_dtc_temperaturas, "matriz_de_confusao_DTC_temperaturas", tamanho_mc_temperaturas)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), dpi=300)
plot_tree(dtc_plot_temperaturas, feature_names=feature_names_temperaturas, class_names=[str(num) for num in temperaturas], max_depth=5, fontsize=6, ax=ax)

plt.savefig("./docs/arvore_de_decisao_temperaturas.png", dpi=300, bbox_inches="tight")

### Floresta aleatória

In [ ]:
rfc_temperaturas = RandomForestClassifier(n_jobs=2, random_state=0)
rfc_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_rfc_temperaturas = rfc_temperaturas.predict(X_test_temperaturas)

classificacao_rfc_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_rfc_temperaturas)

relatorio["random_forest_classifier"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_rfc_temperaturas)),
    "classificacao_temperaturas": classificacao_rfc_temperaturas
})

pd.DataFrame(classificacao_rfc_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_rfc_temperaturas, "matriz_de_confusao_RFC_temperaturas", tamanho_mc_temperaturas)

In [ ]:
fig, ax = plt.subplots(ncols=len(temperaturas), figsize=(90, 20), dpi=300)

for i in range(0, len(temperaturas)):
    plot_tree(rfc_temperaturas.estimators_[i], feature_names=feature_names_temperaturas, class_names=[str(num) for num in temperaturas], max_depth=3, fontsize=6, ax=ax[i])

    ax[i].set_title("Estimador: " + str(i), fontsize=10)

plt.savefig("./docs/floresta_aleatoria_temperaturas.png", dpi=300, bbox_inches="tight")

### Multi-layer Perceptron (ADAM)

In [ ]:
mlpc_adam_temperaturas = MLPClassifier(hidden_layer_sizes=(20,), max_iter=500, activation="relu")
mlpc_adam_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_mlpc_adam_temperaturas = mlpc_adam_temperaturas.predict(X_test_temperaturas)

classificacao_mlpc_adam_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_mlpc_adam_temperaturas)

relatorio["mlp_classifier_adam"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_mlpc_adam_temperaturas)),
    "classificacao_temperaturas": classificacao_mlpc_adam_temperaturas
})

pd.DataFrame(classificacao_mlpc_adam_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_mlpc_adam_temperaturas, "matriz_de_confusao_MLPC_adam_temperaturas", tamanho_mc_temperaturas)

In [ ]:
plt.plot(mlpc_adam_temperaturas.loss_curve_)
plt.title("Perda do modelo")
plt.ylabel("Perda")
plt.xlabel("Iterações")
plt.legend(["Treino"], loc="upper left")
plt.savefig("./docs/perda_do_modelo_MLCP_adam_temperaturas.png", dpi=300, bbox_inches="tight")

### Multi-layer Perceptron (LBFGS)

In [ ]:
mlpc_lbfgs_temperaturas = MLPClassifier(alpha=0, max_iter=1000, solver="lbfgs")
mlpc_lbfgs_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_mlpc_lbfgs_temperaturas = mlpc_lbfgs_temperaturas.predict(X_test_temperaturas)

classificacao_mlpc_lbfgs_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_mlpc_lbfgs_temperaturas)

relatorio["mlp_classifier_lbfgs"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_mlpc_lbfgs_temperaturas)),
    "classificacao_temperaturas": classificacao_mlpc_lbfgs_temperaturas
})

pd.DataFrame(classificacao_mlpc_lbfgs_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_mlpc_lbfgs_temperaturas, "matriz_de_confusao_MLPC_lbfgs_temperaturas", tamanho_mc_temperaturas)

### SVM - Support Vector Machine (RBF)

In [ ]:
svc_rbf_temperaturas = SVC(C=50)
svc_rbf_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_svc_rbf_temperaturas = svc_rbf_temperaturas.predict(X_test_temperaturas)

classificacao_svc_rbf_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_svc_rbf_temperaturas)

relatorio["support_vector_machine_rbf"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_svc_rbf_temperaturas)),
    "classificacao_temperaturas": classificacao_svc_rbf_temperaturas
})

pd.DataFrame(classificacao_svc_rbf_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_svc_rbf_temperaturas, "matriz_de_confusao_SVC_rbf_temperaturas", tamanho_mc_temperaturas)

### SVM - Support Vector Machine (Linear)

In [ ]:
svc_linear_temperaturas = SVC(kernel="linear")
svc_linear_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_svc_linear_temperaturas = svc_linear_temperaturas.predict(X_test_temperaturas)

classificacao_svc_linear_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_svc_linear_temperaturas)

relatorio["support_vector_machine_linear"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_svc_linear_temperaturas)),
    "classificacao_temperaturas": classificacao_svc_linear_temperaturas
})

pd.DataFrame(classificacao_svc_linear_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_svc_linear_temperaturas, "matriz_de_confusao_SVC_linear_temperaturas", tamanho_mc_temperaturas)

### K Neighbors

In [ ]:
knn_temperaturas = make_pipeline(StandardScaler(), KNeighborsClassifier())
knn_temperaturas.fit(X_train_temperaturas, y_train_temperaturas)

resultado_knn_temperaturas = knn_temperaturas.predict(X_test_temperaturas)

classificacao_knn_temperaturas = monta_classificacoes(y_test_temperaturas, resultado_knn_temperaturas)

relatorio["k_neighbors_classifier"].update({
    "acuracia_temperaturas": formata_percentual(accuracy_score(y_test_temperaturas, resultado_knn_temperaturas)),
    "classificacao_temperaturas": classificacao_knn_temperaturas
})

pd.DataFrame(classificacao_knn_temperaturas)

In [ ]:
salva_matriz_de_confusao(y_test_temperaturas, resultado_knn_temperaturas, "matriz_de_confusao_KNN_temperaturas", tamanho_mc_temperaturas)

## Criação de relatório com os resultados

Ao final do treinamento e validação dos modelos, é possível gerar um relatório com os dados obtidos usando um template já criado. Nesse relatório, estarão contidos todas as métricas apresentadas, além da matriz de confusão.

In [ ]:
doc = DocxTemplate("./docs/relatorio_template.docx")

relatorio["logistic_regression"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_LR_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_LR_condicoes.png") else "Imagem não disponível"
relatorio["decision_tree_classifier"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_DTC_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_DTC_condicoes.png") else "Imagem não disponível"
relatorio["random_forest_classifier"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_RFC_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_RFC_condicoes.png") else "Imagem não disponível"
relatorio["mlp_classifier_adam"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_MLPC_adam_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_MLPC_adam_condicoes.png") else "Imagem não disponível"
relatorio["mlp_classifier_lbfgs"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_MLPC_lbfgs_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_MLPC_lbfgs_condicoes.png") else "Imagem não disponível"
relatorio["support_vector_machine_rbf"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_SVC_rbf_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_SVC_rbf_condicoes.png") else "Imagem não disponível"
relatorio["support_vector_machine_linear"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_SVC_linear_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_SVC_linear_condicoes.png") else "Imagem não disponível"
relatorio["k_neighbors_classifier"]["matriz_de_confusao_condicoes"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_KNN_condicoes.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_KNN_condicoes.png") else "Imagem não disponível"

relatorio["logistic_regression"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_LR_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_LR_temperaturas.png") else "Imagem não disponível"
relatorio["decision_tree_classifier"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_DTC_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_DTC_temperaturas.png") else "Imagem não disponível"
relatorio["random_forest_classifier"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_RFC_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_RFC_temperaturas.png") else "Imagem não disponível"
relatorio["mlp_classifier_adam"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_MLPC_adam_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_MLPC_adam_temperaturas.png") else "Imagem não disponível"
relatorio["mlp_classifier_lbfgs"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_MLPC_lbfgs_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_MLPC_lbfgs_temperaturas.png") else "Imagem não disponível"
relatorio["support_vector_machine_rbf"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_SVC_rbf_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_SVC_rbf_temperaturas.png") else "Imagem não disponível"
relatorio["support_vector_machine_linear"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_SVC_linear_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_SVC_linear_temperaturas.png") else "Imagem não disponível"
relatorio["k_neighbors_classifier"]["matriz_de_confusao_temperaturas"] = InlineImage(doc, image_descriptor="./docs/matriz_de_confusao_KNN_temperaturas.png", width=Mm(75), height=Mm(75)) if os.path.exists("./docs/matriz_de_confusao_KNN_temperaturas.png") else "Imagem não disponível"

doc.render(relatorio)
doc.save("./docs/relatorio.docx")

convert("./docs/relatorio.docx", f"./docs/relatorio_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.pdf")

## Backup

Caso seja necessário realizar um backup das informações salvas em arquivos JSON poderá ser executado o comando a seguir, porém garantindo que as informações abaixo estão corretas no *.env*.
- **BACKUP_DB** - Ajustar a variável para permitir a cópia
- **BACKUP_DIR** - Incluir um diretório válido

In [ ]:
if BACKUP_DB:
   for item in data:
      with open(f"{BACKUP_DIR}/{item["_id"]}.json", "w") as arquivo:
         del item["_id"]

         json.dump(item, arquivo)